In [12]:
import pandas as pd
import datetime
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

def train_model_and_predict(transactions_count, toPredictDates):

	def extract_features(dates):
		return pd.DataFrame({
			"timestamp": [int(datetime.datetime.timestamp(x)) for x in dates],
			# "hour": [x.hour for x in dates],
			"weekday": [x.weekday() for x in dates],
			"is_weekend": [1 if x.weekday() >= 5 else 0 for x in dates]
		})

	# Extract features
	X = extract_features(transactions_count["Date"])
	y = transactions_count["TransactionsCount"].values

	# Define and train the model
	model = Pipeline([
		("poly", PolynomialFeatures(degree=2)),
		("scaler", StandardScaler()),  # Add scaling
		("rf", RandomForestRegressor(n_estimators=100, random_state=42))
	])
	
	model.fit(X, y)

	
	X_new = extract_features(toPredictDates)
	Y_new = model.predict(X_new)
		
	print({
		"X": X.to_dict(),
		"y": y.tolist(),
		"X_new": X_new.to_dict(),
		"Y_new": Y_new.tolist()
	})
	return Y_new

# Example usage
transactions_count = pd.DataFrame({
	"Date": [datetime.datetime(2024, 12, 5, 16, 0), datetime.datetime(2024, 12, 5, 16, 5),
	datetime.datetime(2024, 12, 5, 16, 10), datetime.datetime(2024, 12, 5, 16, 15),
	datetime.datetime(2024, 12, 5, 16, 20), datetime.datetime(2024, 12, 5, 16, 25),
	datetime.datetime(2024, 12, 5, 16, 30), datetime.datetime(2024, 12, 5, 16, 35),
	datetime.datetime(2024, 12, 5, 16, 40), datetime.datetime(2024, 12, 5, 16, 45)],
	"TransactionsCount": [1, 2, 0, 1, 0, 1, 1, 1, 1, 1]
})
toPredictDates = [datetime.datetime(2024, 12, 5, 16, 0), datetime.datetime(2024, 12, 5, 16, 5)]

predictions = train_model_and_predict(transactions_count, toPredictDates)
print("Predictions:", predictions)

{'X': {'timestamp': {0: 1733410800, 1: 1733411100, 2: 1733411400, 3: 1733411700, 4: 1733412000, 5: 1733412300, 6: 1733412600, 7: 1733412900, 8: 1733413200, 9: 1733413500}, 'weekday': {0: 3, 1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3, 9: 3}, 'is_weekend': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}}, 'y': [1, 2, 0, 1, 0, 1, 1, 1, 1, 1], 'X_new': {'timestamp': {0: 1733410800, 1: 1733411100}, 'weekday': {0: 3, 1: 3}, 'is_weekend': {0: 0, 1: 0}}, 'Y_new': [1.1, 1.39]}
Predictions: [1.1  1.39]
